In [180]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import tenseal as ts



In [181]:
# Load the heart disease data
heart_data = pd.read_csv('heart_disease_data.csv')

# Preprocessing
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

scaler = StandardScaler()
scaler.fit(X)
standardized_data = scaler.transform(X)
X = standardized_data

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, stratify=Y, random_state=2)

# Train the model
model = svm.SVC(kernel='linear')
model.fit(X_train, Y_train)

SVC(kernel='linear')

In [182]:
# Perform predictions on the training set
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score on training data: ', training_data_accuracy)


Accuracy score on training data:  0.856353591160221


In [183]:

# Perform predictions on the testing set
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score on testing data: ', test_data_accuracy)


Accuracy score on testing data:  0.819672131147541


In [184]:
# Now, let's perform somewhat homomorphic encryption on the model and data using the BFV scheme
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=8192, plain_modulus=1032193, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()


In [185]:
# Encrypt the model coefficients
enc_model_coef = ts.bfv_vector(context, model.coef_[0].tolist())

In [186]:
# Perform encrypted prediction
enc_predictions = [enc_X_test[i].dot(enc_model_coef) for i in range(len(X_test))]

In [187]:
# Decrypt the predictions
decrypted_predictions = [vec.decrypt() for vec in enc_predictions]

In [188]:
# Convert continuous predictions to binary (0 or 1)
threshold = 0.54
binary_predictions = [1 if pred[0] >= threshold else 0 for pred in decrypted_predictions]

# Evaluate accuracy on encrypted predictions
accuracy_on_encrypted_data = accuracy_score(binary_predictions, Y_test)
print('Accuracy score on encrypted testing data: ', accuracy_on_encrypted_data)


Accuracy score on encrypted testing data:  0.5655737704918032
